In [ ]:
import pymongo

In [ ]:
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb= myclient["mydatabase"]

In [ ]:
print(myclient.list_database_names())

In [ ]:
dblist = myclient.list_database_names()
if "mydabase" in dblist:
        print("The database exists.")

In [ ]:
mycol= mydb["customers"]

In [ ]:
print(mydb.list_collection_names())

In [ ]:
mydict = {"name":"John", "address" : "Highway 37"}
x= mycol.insert_one(mydict)


In [ ]:
mydict = {"name":"Peter", "address" : "Lowstreet 27"}
x= mycol.insert_one(mydict)
print(x.inserted_id)

In [ ]:
mylist = [
    {"name":"Amy","address":"Apple st 652"},
    {"name":"Hannah","address":"Mountain 21"},
    {"name":"Michael","address":"Valley 345"},
    {"name":"Sandy","address":"Ocean blvd 2"},
    {"name":"Betty","address":"Green Grass 1"},
    {"name":"Richard","address":"Sky st 331"},
    {"name":"Susan","address":"One way 98"},
    {"name":"Vicky","address":"Yellow Garden 2"},
    {"name":"Ben","address":"Park Lane 38"},
    {"name":"William","address":"Central st 954"},
    {"name":"Chuck","address":"Main Road 989"},
    {"name":"Viola","address":"Sideway 1633"}
]
x =  mycol.insert_many(mylist)


In [ ]:
mylist = [
    {"_id":1,"name":"John","address":"Highway 37"},
    {"_id":2,"name":"Peter","address":"Lowstreet 27"},
    {"_id":3,"name":"Amy","address":"Apple st 652"},
    {"_id":4,"name":"Hannah","address":"Mountain 21"},
    {"_id":5,"name":"Michael","address":"Valley 345"},
    {"_id":6,"name":"Sandy","address":"Ocean blvd 2"},
    {"_id":7,"name":"Betty","address":"Green Grass 1"},
    {"_id":8,"name":"Richard","address":"Sky st 331"},
    {"_id":9,"name":"Susan","address":"One way 98"},
    {"_id":10,"name":"Vicky","address":"Yellow Garden 2"},
    {"_id":11,"name":"Ben","address":"Park Lane 38"},
    {"_id":12,"name":"William","address":"Central st 954"}]
x = mycol.insert_many(mylist)


In [ ]:
x= mycol.find_one()
print(x)

In [ ]:
for x in mycol.find():
    print(x)

In [ ]:
for x in mycol.find({},{"_id":0,"name":1,"address":1}):
    print(x)


In [ ]:
myquery={"address": "Park Lane 38"}
mydoc = mycol.find(myquery)

for x in mydoc:
    print(x)

In [ ]:
myquery={"address": {"$gt":"S"}}
mydoc = mycol.find(myquery)

for x in mydoc:
    print(x)

In [ ]:
mydoc = mycol.find().sort("name")

for x in mydoc:
    print(x)

In [ ]:
myquery={"address": "Mountain 21"}
mycol.delete_one(myquery)


In [ ]:
myquery={"address": {"$gt":"S"}}
x = mycol.delete_many(myquery)
print(x.deleted_count,"documents deleted.")

In [ ]:
myquery={"address": "Valley 345"}
newvalues ={"$set":{"address": "Canyon 123"}}
mycol.update_one(myquery,newvalues)
for x in mycol.find():
    print(x)

In [ ]:
myresult = mycol.find().limit(5)
for x in myresult:
    print(x)

In [ ]:
from paho.mqtt import client as mqtt_client
import random 
import pygame
import time
import json

# Define the IP address and port for broadcasting
broker = '192.168.0.110'#'192.168.0.108'  # Broadcast to all devices on the network
port = 1883
topic = "gamepad/joystick"
cont = 1

# Generate a Client ID with the publish prefix.
client_id = f'ImageProvider-{random.randint(0, 1000)}'
# username = 'emqx'
# password = 'public'

################################################################################
def connect_mqtt():
    def on_connect(client, userdata, flags, rc):
        if rc == 0:
            print("Connected to MQTT Broker!")
        else:
            print("Failed to connect, return code %d\n", rc)

    client = mqtt_client.Client(mqtt_client.CallbackAPIVersion.VERSION1,client_id)
    # client.username_pw_set(username, password)
    client.on_connect = on_connect
    client.connect(broker, port)
    return client
##################################################################################

def publish(client, msg):
    msg_count = 1
    #print(msg)
    result = client.publish(topic, msg) # result: [0, 1]
    status = result[0]
    if status == 0:
        msg_count += 1
        #print("Message Printed")
    else:
        print(f"Failed to send message to topic {topic}")
    
    
##################################################################################


client = connect_mqtt()
client.loop_start()
# Initialize Pygame
pygame.init()

# Check if there's a joystick connected
if not pygame.joystick.get_count():
    print("Error: No joystick detected.")
    quit()

# Get the first joystick (modify the index for other connected controllers)
joystick = pygame.joystick.Joystick(0)

# Get the name of the joystick
joystick_name = joystick.get_name()
print(f"Connected joystick: {joystick_name}")
running = True
clock = pygame.time.Clock()  # Used for recording timerunning = True
clock.tick(60)  # Update at most 60 times per second


while cont == 1:
    #cont =2
    #publish(client, "Helloooo")
    # Check if there's a joystick connected
    #pygame.init()
    joystick_data = []
    if not pygame.joystick.get_count():
        print("Error: No joystick detected.")
        pygame.quit()
        client.loop_stop()
        #quit()
    pygame.event.get()
    axes = joystick.get_numaxes()
    buttons = joystick.get_numbuttons()

    joystick_info = json.dumps({
        "joystick_id": 1,
        "name": joystick.get_name(),
        "axes": [joystick.get_axis(j) for j in range(axes)],
        "buttons": [joystick.get_button(j) for j in range(buttons)],
        "num_axes": axes,
        "num_buttons": buttons
    })
    #joystick_data.append(joystick_info)
    
    x_axis = joystick.get_axis(0)  # Left thumbstick horizontal (-1 to 1)
    y_axis = joystick.get_axis(1)  # Left thumbstick vertical (-1 to 1)
    publish(client, joystick_data)
    time.sleep(0.1)  # Adjust sleep time as needed


client.loop_stop()
# Quit Pygame
pygame.quit()
#cap.release()
##################################################################################

In [ ]:
from paho.mqtt import client as mqtt_client
import random 
import time
import json
import math 
import numpy as np
# Define the IP address and port for broadcasting
broker = '192.168.0.109'#'192.168.0.108'  # Broadcast to all devices on the network
port = 1883
topic_in = "gamepad/joystick"
topic_out = "motor_velocities"
cont = 1
# Generate a Client ID with the publish prefix.
client_id = f'ImageProvider-{random.randint(0, 1000)}'
# username = 'emqx'
# password = 'public'
MAX_ANG_VELOCITY = 2   # rad/s
MAX_LIN_VELOCITY = 40  # Cm/s
phi = 0 #45 , 0~
theta = np.pi/6 #30
ang150 = np.pi*(5/6)
ang90 = np.pi/2
L = 12.5 # CM lenght from center of the platform to the center of the wheel 
R = 5.8/2 # Wheel Radius in cm
coord1 = (L, 0)
coord2 = (-L*math.sin(theta) , L*math.cos(theta))
coord3 = (-L*math.sin(theta) , -L*math.cos(theta))
A = np.array([1, math.tan(phi)])   
B = np.array([[math.cos(ang90), math.sin(ang90)], [-math.sin(ang90),math.cos(ang90)]])
C = np.array([[-coord1[1],1,0],[coord1[0],0,1]])
H01 = (A.dot(B)).dot(C)
#print(H01)
D = np.array([1, math.tan(phi)])   
E = np.array([[math.cos(-ang150),math.sin(-ang150)],[-math.sin(-ang150),math.cos(-ang150)]])
F = np.array([[-coord2[1],1,0],[coord2[0],0,1]])
H02 = (D.dot(E)).dot(F)
G = np.array([1, math.tan(phi)])   
I = np.array([[math.cos(-theta),math.sin(-theta)],[-math.sin(-theta),math.cos(-theta)]])
J = np.array([[-coord3[1],1,0],[coord3[0],0,1]])
H03 = (G.dot(I)).dot(J)
print(H01)
print(H02)
print(H03)
################################################################################
def connect_mqtt():
    def on_connect(client, userdata, flags, rc):
        if rc == 0:
            print("Connected to MQTT Broker!")
        else:
            print("Failed to connect, return code %d\n", rc)

    client = mqtt_client.Client(mqtt_client.CallbackAPIVersion.VERSION1,client_id)
    # client.username_pw_set(username, password)
    client.on_connect = on_connect
    client.on_message = on_message
    client.connect(broker, port)
    client.subscribe(topic_in) 
    return client
##################################################################################

def publish(client, msg):
    msg_count = 1
    #print(msg)
    result = client.publish(topic_out, msg) # result: [0, 1]
    status = result[0]
    if status == 0:
        msg_count += 1
        #print("Message Printed")
    else:
        print(f"Failed to send message to topic {topic}")
    
    
##################################################################################
################################################################################
def on_message(client, userdata, msg):
    try:
        message = msg.payload.decode()
        data = json.loads(message)
        #print(data)
        x_value = (float(data["axes"][0]))
        y_value = (float(data["axes"][1]))
        w_value = (float(data["axes"][2]))
        #print(f"Received joystick data: x={y_value}, y={x_value}, w={w_value}")
        
        Vrx =  -y_value * MAX_LIN_VELOCITY #max 80 cm/s 
        Vry =  x_value * MAX_LIN_VELOCITY
        Wr =   w_value * MAX_ANG_VELOCITY
        vector = np.array([[Wr],[Vrx],[Vry]])
        Vma1 = np.array([H01,H02,H03]).dot(vector)
        Vma1.astype(np.float32)
        Vma=Vma1[0][0]/R        
        Vmb=Vma1[1][0]/R 
        Vmc=Vma1[2][0]/R 
        if abs(Vma) <= 1:
            Vma = 0
        if abs(Vmb) <= 1:
            Vmb = 0
        if abs(Vmc) <= 1:
            Vmc = 0
        Vma= "{:.1f}".format(Vma)
        Vmb= "{:.1f}".format(Vmb)
        Vmc= "{:.1f}".format(Vmc) 
        vel_message = str(Vma)+","+str(Vmb)+","+str(Vmc)
        print(vel_message)
        publish(client, vel_message)
    except (json.JSONDecodeError, KeyError, ValueError) as e:
        print(f"Error parsing message: {e}")
        client.loop_stop()


client = connect_mqtt()
client.loop_start()        


In [ ]:
import math 
import numpy as np
phi = 0 #45 , 0~
theta = np.pi/6 #30
ang150 = np.pi*(5/6)
ang90 = np.pi/2
L = 12.5 # CM lenght from center of the platform to the center of the wheel 
R = 5.8/2 # Wheel Radius
coord1 = (L, 0)
coord2 = (-L*math.sin(theta) , L*math.cos(theta))
coord3 = (-L*math.sin(theta) , -L*math.cos(theta))
A = np.array([1, math.tan(phi)])   
B = np.array([[math.cos(ang90), math.sin(ang90)], [-math.sin(ang90),math.cos(ang90)]])
C = np.array([[-coord1[1],1,0],[coord1[0],0,1]])
H01 = (A.dot(B)).dot(C)
#print(H01)
D = np.array([1, math.tan(phi)])   
E = np.array([[math.cos(-ang150),math.sin(-ang150)],[-math.sin(-ang150),math.cos(-ang150)]])
F = np.array([[-coord2[1],1,0],[coord2[0],0,1]])
H02 = (D.dot(E)).dot(F)
G = np.array([1, math.tan(phi)])   
I = np.array([[math.cos(-theta),math.sin(-theta)],[-math.sin(-theta),math.cos(-theta)]])
J = np.array([[-coord3[1],1,0],[coord3[0],0,1]])
H03 = (G.dot(I)).dot(J)
print(H01)
print(H02)
print(H03)